In [2]:
import os
import datetime
import glob
import h5py
import numpy as np
import xarray as xr
from datetime import datetime
import pandas as pd
import re
import netCDF4 as nc

In [3]:
import dask
import matplotlib.pyplot as plt

In [4]:
import rasterio as rio
import geopandas as gpd
import math

In [5]:
from shutil import copyfile
import gc

###### RENAMING THE FILES CODE

<!-- #Function to calculate the week number of the month
def get_week_of_month(year, month, day):
    first_day = datetime.date(year, month, 1)
    week_number = (day - 1) // 7 + 1  # Calculate week number
    return week_number

#Directory containing the SST files
directory = r"C:\Users\Admin\RIYA PROJECT\DATASETS\mosdac sst hd5

#Loop through files in the directory
for filename in os.listdir(directory):
    if filename.endswith(".h5"):
        # Extract date components from filename
        parts = filename.split("_")
        day = int(parts[1][:2])  # First 2 digits of date
        month_str = parts[1][2:5]  # Next 3 letters (Month)
        year = int(parts[1][5:])  # Remaining digits (Year)
        time = parts[2]  # Time component (0000 or 0015)

        # Convert month abbreviation to number
        month = datetime.datetime.strptime(month_str, "%b").month

        # Get the week number
        week = get_week_of_month(year, month, day)

        # New filename format
        new_filename = f"Week-{week}-{month_str}-{year}-{time}.h5"

        # Rename the file
        old_path = os.path.join(directory, filename)
        new_path = os.path.join(directory, new_filename)
        os.rename(old_path, new_path)
        print(f"Renamed: {filename} → {new_filename}")

print("Renaming completed.") -->

###### CONVERT HD5 FILES TO NC FILES CODE

<!-- def extract_date_from_filename(filename):
    """
    Extract date information from the filename pattern 'Week-X-MMM-YYYY-HHMM'
    Returns a datetime object
    """
    # Extract components using regex
    pattern = r'Week-(\d+)-(\w+)-(\d{4})-(\d{4})'
    match = re.search(pattern, filename)

    if not match:
        raise ValueError(f"Filename {filename} does not match expected pattern")

    week, month, year, time_str = match.groups()

    # Convert month abbreviation to number
    month_dict = {
        'JAN': 1, 'FEB': 2, 'MAR': 3, 'APR': 4, 'MAY': 5, 'JUN': 6,
        'JUL': 7, 'AUG': 8, 'SEP': 9, 'OCT': 10, 'NOV': 11, 'DEC': 12
    }
    month_num = month_dict.get(month.upper())

    # Extract hours and minutes
    hours = int(time_str[:2])
    minutes = int(time_str[2:])

    # Create a datetime object (using day 1 since specific day is not in filename)
    # For a more accurate date, you might want to calculate based on week number
    return datetime(int(year), month_num, 1, hours, minutes)

def h5_to_nc(input_file, output_file):
    """
    Convert an H5 file to NetCDF format
    Handles different key structures based on timestamps
    """
    print(f"Converting {input_file} to {output_file}")

    # Open the H5 file
    with h5py.File(input_file, 'r') as h5_data:
        # Check available keys
        keys = list(h5_data.keys())
        print(f"Found keys: {keys}")

        # Extract date from filename
        date_time = extract_date_from_filename(os.path.basename(input_file))

        # Create dictionary to store data
        data_dict = {}

        # Read all variables from H5 file
        for key in keys:
            data_dict[key] = h5_data[key][:]

        # Create xarray Dataset
        ds = xr.Dataset()

        # Add coordinates
        ds.coords['time'] = date_time

        # Assume latitude and longitude are 2D arrays
        if 'Latitude' in data_dict and 'Longitude' in data_dict:
            lat_shape = data_dict['Latitude'].shape
            lon_shape = data_dict['Longitude'].shape

            # If they're 2D arrays, we need a different approach
            if len(lat_shape) == 2 and len(lon_shape) == 2:
                # Create y, x dimensions
                ds.coords['y'] = np.arange(lat_shape[0])
                ds.coords['x'] = np.arange(lat_shape[1])

                # Add lat/lon as variables instead of coordinates
                ds['Latitude'] = (('y', 'x'), data_dict['Latitude'])
                ds['Longitude'] = (('y', 'x'), data_dict['Longitude'])
            else:
                # Use them as coordinates if they're 1D
                ds.coords['latitude'] = data_dict['Latitude']
                ds.coords['longitude'] = data_dict['Longitude']

        # Add variables to the dataset
        for key in keys:
            if key not in ['Latitude', 'Longitude']:  # These were already handled
                if key == 'time':
                    continue  # Skip since we created a proper time coordinate

                # Handle different dimensions based on the variable shape
                var_shape = data_dict[key].shape

                # Define dimensions based on shape
                if len(var_shape) == 2:
                    ds[key] = (('y', 'x'), data_dict[key])
                elif len(var_shape) == 1:
                    if var_shape[0] == lat_shape[0]:
                        ds[key] = (('y'), data_dict[key])
                    elif var_shape[0] == lat_shape[1]:
                        ds[key] = (('x'), data_dict[key])
                    else:
                        # For other 1D arrays
                        ds[key] = (('index'), data_dict[key])
                else:
                    # For other dimensionalities
                    dim_names = [f'dim_{i}' for i in range(len(var_shape))]
                    ds[key] = (tuple(dim_names), data_dict[key])

        # Add SST_QFLAGS_WKL if it doesn't exist in the original file
        # (for files with timestamp 0015)
        if 'SST_QFLAGS_WKL' not in keys and 'SST_WKL' in keys:
            # Create a default quality flag (all good)
            ds['SST_QFLAGS_WKL'] = ds['SST_WKL'] * 0

        # Add metadata
        ds.attrs['title'] = 'MOSDAC SST Data'
        ds.attrs['source_file'] = os.path.basename(input_file)
        ds.attrs['created'] = datetime.now().isoformat()

        # Save to NetCDF
        ds.to_netcdf(output_file)
        print(f"Successfully saved {output_file}")

def convert_all_files(input_dir, output_dir):
    """
    Convert all H5 files in input_dir to NetCDF files in output_dir
    """
    # Create output directory if it doesn't exist
    os.makedirs(output_dir, exist_ok=True)

    # Get all H5 files
    h5_files = glob.glob(os.path.join(input_dir, '*.h5'))
    print(f"Found {len(h5_files)} H5 files")

    # Process each file
    for h5_file in h5_files:
        # Create output filename
        base_name = os.path.basename(h5_file)
        nc_file = os.path.join(output_dir, base_name.replace('.h5', '.nc'))

        # Convert file
        try:
            h5_to_nc(h5_file, nc_file)
        except Exception as e:
            print(f"Error processing {h5_file}: {str(e)}")

if __name__ == "__main__":
    # Set input and output directories
    input_directory = r"C:\Users\Admin\RIYA PROJECT\DATASETS\mosdac sst hd5"
    output_directory = r"C:\Users\Admin\RIYA PROJECT\DATASETS\mosdac sst nc"

    # Convert all files
    convert_all_files(input_directory, output_directory) -->

###### REMOVING MISSING VALUES (-999)

In [4]:
import os
import numpy as np
import xarray as xr

def clean_netcdf_files(input_folder, output_folder, fill_value=-999):
    """
    Process NetCDF files in the input folder, replacing specified fill values
    while preserving original dimensions and handling integer data types properly.
    No mask variables are created to ensure compatibility with other datasets.
   
    Parameters:
    -----------
    input_folder : str
        Path to folder containing NetCDF files to process
    output_folder : str
        Path to folder where processed files will be saved
    fill_value : float, optional
        Value to be replaced (default is -999)
    """
    # Create output folder if it doesn't exist
    os.makedirs(output_folder, exist_ok=True)
   
    # Process each NetCDF file in the input folder
    for filename in os.listdir(input_folder):
        if filename.endswith('.nc'):
            # Full path to input and output files
            input_path = os.path.join(input_folder, filename)
            output_path = os.path.join(output_folder, filename)
           
            try:
                print(f"Processing: {filename}")
               
                # Open the NetCDF file
                # Use decode_cf=False to avoid automatic decoding of _FillValue
                ds = xr.open_dataset(input_path, decode_cf=False)
               
                # Create a deep copy of the dataset
                ds_cleaned = ds.copy(deep=True)
               
                # Store original encoding for all variables
                original_encoding = {}
                for var_name in ds.variables:
                    if hasattr(ds[var_name], 'encoding'):
                        original_encoding[var_name] = ds[var_name].encoding.copy()
               
                # Process all data variables (not just specific ones)
                for var_name in ds.data_vars:
                    print(f"  Checking variable: {var_name}")
                   
                    # Get current data type
                    current_dtype = ds[var_name].dtype
                    is_int_dtype = np.issubdtype(current_dtype, np.integer)
                   
                    # Only process the variable if it contains the fill value
                    var_data = ds[var_name].values
                    contains_fill = np.any(var_data == fill_value)
                   
                    if contains_fill:
                        print(f"  Found fill values in {var_name}")
                       
                        # Different handling based on data type
                        if is_int_dtype:
                            # For integer types, we need to handle differently
                            # Use a valid fill value for the integer type instead of creating masks
                            print(f"  {var_name} has integer type {current_dtype}")
                           
                            # Get the appropriate fill value for this data type
                            if np.issubdtype(current_dtype, np.int8):
                                new_fill = np.iinfo(np.int8).min
                            elif np.issubdtype(current_dtype, np.int16):
                                new_fill = np.iinfo(np.int16).min
                            elif np.issubdtype(current_dtype, np.int32):
                                new_fill = np.iinfo(np.int32).min
                            elif np.issubdtype(current_dtype, np.int64):
                                new_fill = np.iinfo(np.int64).min
                            elif np.issubdtype(current_dtype, np.uint8):
                                new_fill = np.iinfo(np.uint8).max
                            elif np.issubdtype(current_dtype, np.uint16):
                                new_fill = np.iinfo(np.uint16).max
                            elif np.issubdtype(current_dtype, np.uint32):
                                new_fill = np.iinfo(np.uint32).max
                            elif np.issubdtype(current_dtype, np.uint64):
                                new_fill = np.iinfo(np.uint64).max
                            else:
                                # Use -9999 as a fallback
                                new_fill = -9999
                           
                            # Replace the fill values with the new fill value
                            cleaned_data = var_data.copy()
                            cleaned_data[cleaned_data == fill_value] = new_fill
                            ds_cleaned[var_name].values = cleaned_data
                           
                            # Make sure the _FillValue in encoding is updated
                            if var_name in original_encoding:
                                if '_FillValue' in original_encoding[var_name]:
                                    original_encoding[var_name]['_FillValue'] = new_fill
                           
                            print(f"  Replaced fill value with {new_fill} for {var_name}")
                        else:
                            # For float types, we can directly replace with NaN
                            print(f"  {var_name} has float type {current_dtype}, replacing with NaN")
                            cleaned_data = var_data.copy()
                            cleaned_data[cleaned_data == fill_value] = np.nan
                            ds_cleaned[var_name].values = cleaned_data
               
                # Set up encoding for saving
                encoding = {}
                for var_name in ds_cleaned.variables:
                    # Start with compression settings
                    encoding[var_name] = {
                        'zlib': True,
                        'complevel': 3,
                    }
                   
                    # Copy original encoding where available
                    if var_name in original_encoding:
                        # Copy selected attributes from original encoding
                        for key in ['dtype', '_FillValue', 'scale_factor', 'add_offset']:
                            if key in original_encoding[var_name]:
                                encoding[var_name][key] = original_encoding[var_name][key]
               
                # Save the cleaned dataset
                print(f"  Saving to: {output_path}")
                ds_cleaned.to_netcdf(output_path, encoding=encoding)
               
                # Close the datasets
                ds.close()
                ds_cleaned.close()
                print(f"Completed: {filename}\n")
               
            except Exception as e:
                print(f"Error processing {filename}: {str(e)}")

# Example usage
input_folder = r"C:\Users\Admin\RIYA PROJECT\DATASETS\mosdac sst nc files"
output_folder = r"C:\Users\Admin\RIYA PROJECT\DATASETS\mosdac cleaned nc files"

# Call the function
if __name__ == "__main__":
    print("Starting NetCDF clean-up process...")
    clean_netcdf_files(input_folder, output_folder)
    print("Processing complete.")

Starting NetCDF clean-up process...
Processing: Week-1-APR-2019-0000.nc
  Checking variable: time
  Checking variable: Latitude
  Found fill values in Latitude
  Latitude has integer type int16
  Replaced fill value with -32768 for Latitude
  Checking variable: Longitude
  Checking variable: BINNING_COUNTER
  Checking variable: GeoX
  Checking variable: GeoY
  Checking variable: SST_QFLAGS_WKL
  Checking variable: SST_WKL
  Found fill values in SST_WKL
  SST_WKL has float type float32, replacing with NaN
  Saving to: C:\Users\Admin\RIYA PROJECT\DATASETS\mosdac cleaned nc files\Week-1-APR-2019-0000.nc
Completed: Week-1-APR-2019-0000.nc

Processing: Week-1-APR-2020-0000.nc
  Checking variable: time
  Checking variable: Latitude
  Found fill values in Latitude
  Latitude has integer type int16
  Replaced fill value with -32768 for Latitude
  Checking variable: Longitude
  Checking variable: BINNING_COUNTER
  Checking variable: GeoX
  Checking variable: GeoY
  Checking variable: SST_QFLAGS

  Checking variable: GeoX
  Checking variable: GeoY
  Checking variable: SST_QFLAGS_WKL
  Checking variable: SST_WKL
  Found fill values in SST_WKL
  SST_WKL has float type float32, replacing with NaN
  Saving to: C:\Users\Admin\RIYA PROJECT\DATASETS\mosdac cleaned nc files\Week-1-DEC-2019-0000.nc
Completed: Week-1-DEC-2019-0000.nc

Processing: Week-1-DEC-2020-0015.nc
  Checking variable: time
  Checking variable: Latitude
  Found fill values in Latitude
  Latitude has integer type int16
  Replaced fill value with -32768 for Latitude
  Checking variable: Longitude
  Checking variable: BINNING_COUNTER
  Checking variable: GeoX
  Checking variable: GeoY
  Checking variable: SST_WKL
  Found fill values in SST_WKL
  SST_WKL has float type float32, replacing with NaN
  Checking variable: SST_QFLAGS_WKL
  Saving to: C:\Users\Admin\RIYA PROJECT\DATASETS\mosdac cleaned nc files\Week-1-DEC-2020-0015.nc
Completed: Week-1-DEC-2020-0015.nc

Processing: Week-1-DEC-2021-0015.nc
  Checking variable: 

Completed: Week-1-JAN-2020-0015.nc

Processing: Week-1-JAN-2021-0015.nc
  Checking variable: time
  Checking variable: Latitude
  Found fill values in Latitude
  Latitude has integer type int16
  Replaced fill value with -32768 for Latitude
  Checking variable: Longitude
  Checking variable: BINNING_COUNTER
  Checking variable: GeoX
  Checking variable: GeoY
  Checking variable: SST_WKL
  Found fill values in SST_WKL
  SST_WKL has float type float32, replacing with NaN
  Checking variable: SST_QFLAGS_WKL
  Saving to: C:\Users\Admin\RIYA PROJECT\DATASETS\mosdac cleaned nc files\Week-1-JAN-2021-0015.nc
Completed: Week-1-JAN-2021-0015.nc

Processing: Week-1-JAN-2022-0015.nc
  Checking variable: time
  Checking variable: Latitude
  Found fill values in Latitude
  Latitude has integer type int16
  Replaced fill value with -32768 for Latitude
  Checking variable: Longitude
  Checking variable: BINNING_COUNTER
  Checking variable: GeoX
  Checking variable: GeoY
  Checking variable: SST_WKL
  

  Checking variable: GeoX
  Checking variable: GeoY
  Checking variable: SST_WKL
  Found fill values in SST_WKL
  SST_WKL has float type float32, replacing with NaN
  Checking variable: SST_QFLAGS_WKL
  Saving to: C:\Users\Admin\RIYA PROJECT\DATASETS\mosdac cleaned nc files\Week-1-JUN-2021-0015.nc
Completed: Week-1-JUN-2021-0015.nc

Processing: Week-1-JUN-2022-0015.nc
  Checking variable: time
  Checking variable: Latitude
  Found fill values in Latitude
  Latitude has integer type int16
  Replaced fill value with -32768 for Latitude
  Checking variable: Longitude
  Checking variable: BINNING_COUNTER
  Checking variable: GeoX
  Checking variable: GeoY
  Checking variable: SST_WKL
  Found fill values in SST_WKL
  SST_WKL has float type float32, replacing with NaN
  Checking variable: SST_QFLAGS_WKL
  Saving to: C:\Users\Admin\RIYA PROJECT\DATASETS\mosdac cleaned nc files\Week-1-JUN-2022-0015.nc
Completed: Week-1-JUN-2022-0015.nc

Processing: Week-1-JUN-2023-0015.nc
  Checking variable: 

Completed: Week-1-MAY-2023-0015.nc

Processing: Week-1-MAY-2024-0015.nc
  Checking variable: time
  Checking variable: Latitude
  Found fill values in Latitude
  Latitude has integer type int16
  Replaced fill value with -32768 for Latitude
  Checking variable: Longitude
  Checking variable: BINNING_COUNTER
  Checking variable: GeoX
  Checking variable: GeoY
  Checking variable: SST_WKL
  Found fill values in SST_WKL
  SST_WKL has float type float32, replacing with NaN
  Checking variable: SST_QFLAGS_WKL
  Saving to: C:\Users\Admin\RIYA PROJECT\DATASETS\mosdac cleaned nc files\Week-1-MAY-2024-0015.nc
Completed: Week-1-MAY-2024-0015.nc

Processing: Week-1-NOV-2019-0000.nc
  Checking variable: time
  Checking variable: Latitude
  Found fill values in Latitude
  Latitude has integer type int16
  Replaced fill value with -32768 for Latitude
  Checking variable: Longitude
  Checking variable: BINNING_COUNTER
  Checking variable: GeoX
  Checking variable: GeoY
  Checking variable: SST_QFLAGS

...

  Found fill values in Latitude
  Latitude has integer type int16
  Replaced fill value with -32768 for Latitude
  Checking variable: Longitude
  Checking variable: BINNING_COUNTER
  Checking variable: GeoX
  Checking variable: GeoY
  Checking variable: SST_WKL
  Found fill values in SST_WKL
  SST_WKL has float type float32, replacing with NaN
  Checking variable: SST_QFLAGS_WKL
  Saving to: C:\Users\Admin\RIYA PROJECT\DATASETS\mosdac cleaned nc files\Week-4-APR-2024-0015.nc
Completed: Week-4-APR-2024-0015.nc

Processing: Week-4-AUG-2020-0000.nc
  Checking variable: time
  Checking variable: Latitude
  Found fill values in Latitude
  Latitude has integer type int16
  Replaced fill value with -32768 for Latitude
  Checking variable: Longitude
  Checking variable: BINNING_COUNTER
  Checking variable: GeoX
  Checking variable: GeoY
  Checking variable: SST_QFLAGS_WKL
  Checking variable: SST_WKL
  Found fill values in SST_WKL
  SST_WKL has float type float32, replacing with NaN
  Saving t

  Saving to: C:\Users\Admin\RIYA PROJECT\DATASETS\mosdac cleaned nc files\Week-4-FEB-2021-0015.nc
Completed: Week-4-FEB-2021-0015.nc

Processing: Week-4-FEB-2022-0015.nc
  Checking variable: time
  Checking variable: Latitude
  Found fill values in Latitude
  Latitude has integer type int16
  Replaced fill value with -32768 for Latitude
  Checking variable: Longitude
  Checking variable: BINNING_COUNTER
  Checking variable: GeoX
  Checking variable: GeoY
  Checking variable: SST_WKL
  Found fill values in SST_WKL
  SST_WKL has float type float32, replacing with NaN
  Checking variable: SST_QFLAGS_WKL
  Saving to: C:\Users\Admin\RIYA PROJECT\DATASETS\mosdac cleaned nc files\Week-4-FEB-2022-0015.nc
Completed: Week-4-FEB-2022-0015.nc

Processing: Week-4-FEB-2024-0015.nc
  Checking variable: time
  Checking variable: Latitude
  Found fill values in Latitude
  Latitude has integer type int16
  Replaced fill value with -32768 for Latitude
  Checking variable: Longitude
  Checking variable: B

  Checking variable: BINNING_COUNTER
  Checking variable: GeoX
  Checking variable: GeoY
  Checking variable: SST_QFLAGS_WKL
  Checking variable: SST_WKL
  Found fill values in SST_WKL
  SST_WKL has float type float32, replacing with NaN
  Saving to: C:\Users\Admin\RIYA PROJECT\DATASETS\mosdac cleaned nc files\Week-4-JUN-2020-0000.nc
Completed: Week-4-JUN-2020-0000.nc

Processing: Week-4-JUN-2024-0015.nc
  Checking variable: time
  Checking variable: Latitude
  Found fill values in Latitude
  Latitude has integer type int16
  Replaced fill value with -32768 for Latitude
  Checking variable: Longitude
  Checking variable: BINNING_COUNTER
  Checking variable: GeoX
  Checking variable: GeoY
  Checking variable: SST_WKL
  Found fill values in SST_WKL
  SST_WKL has float type float32, replacing with NaN
  Checking variable: SST_QFLAGS_WKL
  Saving to: C:\Users\Admin\RIYA PROJECT\DATASETS\mosdac cleaned nc files\Week-4-JUN-2024-0015.nc
Completed: Week-4-JUN-2024-0015.nc

Processing: Week-4-M

Completed: Week-4-OCT-2021-0015.nc

Processing: Week-4-OCT-2022-0015.nc
  Checking variable: time
  Checking variable: Latitude
  Found fill values in Latitude
  Latitude has integer type int16
  Replaced fill value with -32768 for Latitude
  Checking variable: Longitude
  Checking variable: BINNING_COUNTER
  Checking variable: GeoX
  Checking variable: GeoY
  Checking variable: SST_WKL
  Found fill values in SST_WKL
  SST_WKL has float type float32, replacing with NaN
  Checking variable: SST_QFLAGS_WKL
  Saving to: C:\Users\Admin\RIYA PROJECT\DATASETS\mosdac cleaned nc files\Week-4-OCT-2022-0015.nc
Completed: Week-4-OCT-2022-0015.nc

Processing: Week-4-OCT-2023-0015.nc
  Checking variable: time
  Checking variable: Latitude
  Found fill values in Latitude
  Latitude has integer type int16
  Replaced fill value with -32768 for Latitude
  Checking variable: Longitude
  Checking variable: BINNING_COUNTER
  Checking variable: GeoX
  Checking variable: GeoY
  Checking variable: SST_WKL
  

  Found fill values in SST_WKL
  SST_WKL has float type float32, replacing with NaN
  Checking variable: SST_QFLAGS_WKL
  Saving to: C:\Users\Admin\RIYA PROJECT\DATASETS\mosdac cleaned nc files\Week-5-JUL-2024-0015.nc
Completed: Week-5-JUL-2024-0015.nc

Processing: Week-5-JUN-2020-0000.nc
  Checking variable: time
  Checking variable: Latitude
  Found fill values in Latitude
  Latitude has integer type int16
  Replaced fill value with -32768 for Latitude
  Checking variable: Longitude
  Checking variable: BINNING_COUNTER
  Checking variable: GeoX
  Checking variable: GeoY
  Checking variable: SST_QFLAGS_WKL
  Checking variable: SST_WKL
  Found fill values in SST_WKL
  SST_WKL has float type float32, replacing with NaN
  Saving to: C:\Users\Admin\RIYA PROJECT\DATASETS\mosdac cleaned nc files\Week-5-JUN-2020-0000.nc
Completed: Week-5-JUN-2020-0000.nc

Processing: Week-5-MAR-2020-0000.nc
  Checking variable: time
  Checking variable: Latitude
  Found fill values in Latitude
  Latitude has

In [6]:
a = xr.open_dataset(r"C:\Users\Admin\RIYA PROJECT\DATASETS\mosdac cleaned nc files\Week-1-SEP-2022-0015.nc")
b = np.unique(a['SST_WKL'].values)
print(b)
a

[283.1065  283.1066  283.13193 ... 306.20712 306.22168       nan]


<xarray.Dataset>
Dimensions:          (y: 2816, x: 2805, dim_0: 1, dim_1: 2816, dim_2: 2805)
Coordinates:
    time             datetime64[ns] ...
  * y                (y) int32 0 1 2 3 4 5 6 ... 2810 2811 2812 2813 2814 2815
  * x                (x) int32 0 1 2 3 4 5 6 ... 2799 2800 2801 2802 2803 2804
Dimensions without coordinates: dim_0, dim_1, dim_2
Data variables:
    Latitude         (y, x) int16 ...
    Longitude        (y, x) int16 ...
    BINNING_COUNTER  (dim_0, dim_1, dim_2) int16 ...
    GeoX             (x) int32 ...
    GeoY             (y) int32 ...
    SST_WKL          (dim_0, dim_1, dim_2) float32 nan nan nan ... nan nan nan
    SST_QFLAGS_WKL   (dim_0, dim_1, dim_2) float32 ...
Attributes:
    title:        MOSDAC SST Data
    source_file:  Week-1-SEP-2022-0015.h5
    created:      2025-03-05T11:34:03.013867

###### COMBINING INTO ONE DATASET

In [4]:
def extract_date_from_filename(filename):
    """
    Extract date information from the filename pattern 'Week-X-MMM-YYYY-HHMM'
    Returns a datetime object
    """
    # Extract components using regex
    pattern = r'Week-(\d+)-(\w+)-(\d{4})-(\d{4})'
    match = re.search(pattern, os.path.basename(filename))

    if not match:
        return None

    week, month, year, time_str = match.groups()

    # Convert month abbreviation to number
    month_dict = {
        'JAN': 1, 'FEB': 2, 'MAR': 3, 'APR': 4, 'MAY': 5, 'JUN': 6,
        'JUL': 7, 'AUG': 8, 'SEP': 9, 'OCT': 10, 'NOV': 11, 'DEC': 12
    }
    month_num = month_dict.get(month.upper())
    if not month_num:
        return None

    # Convert week to approximate day (rough estimate)
    day = min((int(week) - 1) * 7 + 1, 28)  # Cap at 28 to avoid invalid dates

    # Extract hours and minutes
    try:
        hours = int(time_str[:2])
        minutes = int(time_str[2:])
    except ValueError:
        return None

    # Create a datetime object
    try:
        return datetime(int(year), month_num, day, hours, minutes)
    except ValueError:
        return None

def combine_nc_files_dask_optimized(input_dir, output_file, chunk_size=None):
    """
    Optimized function to combine NetCDF files using Dask
    """
    print(f"Searching for NetCDF files in {input_dir}")

    # Find all NC files
    nc_files = sorted(glob.glob(os.path.join(input_dir, "*.nc")))

    if not nc_files:
        raise ValueError(f"No NetCDF files found in {input_dir}")

    print(f"Found {len(nc_files)} NetCDF files to combine")
   
    # Pre-process to extract times from all filenames
    file_times = {}
    valid_files = []
   
    for nc_file in nc_files:
        file_time = extract_date_from_filename(nc_file)
        if file_time is not None:
            file_times[nc_file] = file_time
            valid_files.append(nc_file)
        else:
            print(f"Warning: Could not extract time from {os.path.basename(nc_file)}")
   
    if not valid_files:
        raise ValueError("No valid datasets with extractable times were found")
   
    # Sort files by extracted time to ensure chronological order
    valid_files.sort(key=lambda x: file_times[x])
    print(f"Processing {len(valid_files)} valid files")
   
    # Determine optimal chunking by examining the first file
    if chunk_size is None:
        with xr.open_dataset(valid_files[0]) as sample_ds:
            # Get dimensions
            dims = {dim: len(sample_ds[dim]) for dim in sample_ds.dims}
           
            # Calculate chunks based on data size and dimensions
            # For time dimension, use smaller chunks since we're combining many files
            chunks = {}
            for dim, size in dims.items():
                if dim == 'time':
                    chunks[dim] = 1  # Time dimension will grow with concatenation
                elif size > 100:
                    chunks[dim] = size // 4  # Larger dimensions get divided
                else:
                    chunks[dim] = size  # Small dimensions kept whole
    else:
        # Use user-provided chunking
        chunks = chunk_size
   
    print(f"Using chunk size: {chunks}")
   
    # Process files in smaller batches to prevent memory issues
    batch_size = min(50, len(valid_files))  # Adjust batch size based on file count
    combined_ds = None
   
    for i in range(0, len(valid_files), batch_size):
        batch_files = valid_files[i:i+batch_size]
        print(f"Processing batch {i//batch_size + 1} of {(len(valid_files) + batch_size - 1)//batch_size} ({len(batch_files)} files)")
       
        # Open datasets and assign time coordinates
        datasets = []
        for nc_file in batch_files:
            # Open with specified chunks
            ds = xr.open_dataset(nc_file, chunks=chunks, engine='netcdf4')
           
            # Add time coordinate
            ds = ds.expand_dims(dim={"time": [file_times[nc_file]]})
            datasets.append(ds)
       
        # Combine this batch
        batch_ds = xr.concat(datasets, dim='time')
       
        # Close file handles in this batch
        for ds in datasets:
            ds.close()
       
        # Append to existing combined dataset or create new one
        if combined_ds is None:
            combined_ds = batch_ds
        else:
            combined_ds = xr.concat([combined_ds, batch_ds], dim='time')
            batch_ds.close()  # Close batch dataset after concatenation
   
    # Sort by time
    combined_ds = combined_ds.sortby('time')
   
    # Add metadata
    combined_ds.attrs['title'] = 'Combined MOSDAC SST Data (Dask-optimized)'
    combined_ds.attrs['created'] = datetime.now().isoformat()
    combined_ds.attrs['source_file_count'] = len(valid_files)
   
    # Set up encoding for compressed output
    encoding = {var: {'zlib': True, 'complevel': 1} for var in combined_ds.data_vars}
   
    # Write to netCDF with explicit chunking for output
    print(f"Saving combined dataset to {output_file}")
    combined_ds.to_netcdf(
        output_file,
        encoding=encoding,
        compute=True,
        engine='netcdf4',
        unlimited_dims=['time']  # Makes time dimension expandable
    )
   
    # Calculate memory estimate without loading all data
    total_size_mb = sum(
        v.nbytes for v in combined_ds.values()
    ) / (1024 * 1024)
   
    print(f"Successfully created combined dataset with {len(combined_ds.time)} time points")
    print(f"Estimated dataset size: {total_size_mb:.2f} MB")
   
    # Return dataset for inspection (but load minimally)
    return combined_ds

if __name__ == "__main__":
    # Set Dask memory limit
    dask.config.set({"array.slicing.split_large_chunks": False})
    dask.config.set({"distributed.worker.memory.target": 0.8})  # Use up to 80% of memory
    dask.config.set({"distributed.worker.memory.spill": 0.9})   # Spill to disk at 90%

    input_directory = r"C:\Users\Admin\RIYA PROJECT\DATASETS\mosdac cleaned nc files"
    output_file = r"C:\Users\Admin\RIYA PROJECT\DATASETS\mosdac_sst_cleaned.nc" 

    # Combine all files using optimized Dask approach
    # Option 1: Let the function determine optimal chunking
    combined_dataset = combine_nc_files_dask_optimized(input_directory, output_file)
   
    # Option 2: Specify custom chunking if needed
    # combined_dataset = combine_nc_files_dask_optimized(
    #     input_directory, output_file,
    #     chunk_size={'lat': 50, 'lon': 50, 'time': 1}
    # )
   
    # Print minimal information (avoid loading full dataset into memory)
    print("\nCombined Dataset Information:")
    print(f"Time range: {combined_dataset.time.values[0]} to {combined_dataset.time.values[-1]}")
    print(f"Number of time points: {len(combined_dataset.time)}")
    print(f"Variables: {list(combined_dataset.variables)}")

Searching for NetCDF files in C:\Users\Admin\RIYA PROJECT\DATASETS\mosdac cleaned nc files
Found 278 NetCDF files to combine
Processing 278 valid files
Using chunk size: {'y': 704, 'x': 701, 'dim_0': 1, 'dim_1': 704, 'dim_2': 701}
Processing batch 1 of 6 (50 files)
Processing batch 2 of 6 (50 files)
Processing batch 3 of 6 (50 files)
Processing batch 4 of 6 (50 files)
Processing batch 5 of 6 (50 files)
Processing batch 6 of 6 (28 files)
Saving combined dataset to C:\Users\Admin\RIYA PROJECT\DATASETS\mosdac_sst_cleaned.nc
Successfully created combined dataset with 278 time points
Estimated dataset size: -28019.76 MB

Combined Dataset Information:
Time range: 2019-01-01T00:00:00.000000000 to 2025-01-01T00:15:00.000000000
Number of time points: 278
Variables: ['time', 'y', 'x', 'Latitude', 'Longitude', 'BINNING_COUNTER', 'GeoX', 'GeoY', 'SST_QFLAGS_WKL', 'SST_WKL']


In [5]:
c= xr.open_dataset(r"C:\Users\Admin\RIYA PROJECT\DATASETS\mosdac_sst_cleaned.nc")
c

<xarray.Dataset>
Dimensions:          (time: 278, y: 2816, x: 2805, dim_0: 1, dim_1: 2816, dim_2: 2805)
Coordinates:
  * time             (time) datetime64[ns] 2019-01-01 ... 2025-01-01T00:15:00
  * y                (y) int32 0 1 2 3 4 5 6 ... 2810 2811 2812 2813 2814 2815
  * x                (x) int32 0 1 2 3 4 5 6 ... 2799 2800 2801 2802 2803 2804
Dimensions without coordinates: dim_0, dim_1, dim_2
Data variables:
    Latitude         (time, y, x) int16 ...
    Longitude        (time, y, x) int16 ...
    BINNING_COUNTER  (time, dim_0, dim_1, dim_2) int16 ...
    GeoX             (time, x) int32 ...
    GeoY             (time, y) int32 ...
    SST_QFLAGS_WKL   (time, dim_0, dim_1, dim_2) float32 ...
    SST_WKL          (time, dim_0, dim_1, dim_2) float32 ...
Attributes:
    title:              Combined MOSDAC SST Data (Dask-optimized)
    source_file:        Week-1-JAN-2019-0000.h5
    created:            2025-04-01T11:38:50.536703
    source_file_count:  278

In [8]:
# #DROPPING THE VARIABLES GeoY , GeoX , BINNING_COUNTER, SST_QFLAGS_WKL FROM DATASET

# from shutil import copyfile

# def remove_variables_from_netcdf(input_file, variables_to_remove, create_backup=True):
#     """
#     Removes specified variables from a NetCDF file and saves changes to the same file.
#     Parameters:
#     input_file : str
#         Path to the NetCDF file to modify
#     variables_to_remove : list
#         List of variable names to remove from the NetCDF file
#     create_backup : bool, optional
#         Whether to create a backup of the original file before modifying (default: True)

#     """
#     # Create backup if requested
#     if create_backup:
#         backup_file = input_file + ".backup"
#         if not os.path.exists(backup_file):
#             print(f"Creating backup at {backup_file}")
#             copyfile(input_file, backup_file)

#     # Load the NetCDF file using xarray
#     ds = xr.open_dataset(input_file)

#     # Print variables before deletion
#     print(f"Variables before deletion: {list(ds.variables)}")

#     # Check which variables from the list exist in the dataset
#     existing_vars = [var for var in variables_to_remove if var in ds.variables]
#     non_existing_vars = [var for var in variables_to_remove if var not in ds.variables]

#     if non_existing_vars:
#         print(f"Warning: The following variables do not exist in the dataset: {non_existing_vars}")

#     if not existing_vars:
#         print("None of the specified variables exist in the dataset. No changes made.")
#         ds.close()
#         return False

#     # Create a new dataset without the specified variables
#     ds_new = ds.drop_vars(existing_vars)

#     # Load dataset into memory before closing original file
#     ds_new.load()

#     # Close the original dataset
#     ds.close()

#     # Save to a temporary file
#     temp_file = input_file + ".temp"
#     print(f"Saving modified dataset...")
#     ds_new.to_netcdf(temp_file)
#     ds_new.close()  # Close new dataset to release any locks

#     # Replace the original file
#     os.remove(input_file)
#     os.rename(temp_file, input_file)

#     print(f"Successfully updated {input_file}")
#     return True

# #Usage example
# if __name__ == "__main__":
#     input_file = (r"C:\Users\Admin\RIYA PROJECT\DATASETS\mosdac_sst_cleaned.nc")
#     variables_to_remove = ["GeoY", "GeoX", "BINNING_COUNTER", "SST_QFLAGS_WKL"]

#     success = remove_variables_from_netcdf(input_file, variables_to_remove)

#     if success:
#         print("Variables were successfully removed from the file.")
#     else:
#         print("Operation cancelled. No changes were made.")

In [45]:
df= xr.open_dataset(r"C:\Users\Admin\RIYA PROJECT\DATASETS\mosdac_sst_cleaned.nc")

In [46]:
df

<xarray.Dataset>
Dimensions:    (time: 278, y: 2816, x: 2805, dim_0: 1, dim_1: 2816, dim_2: 2805)
Coordinates:
  * time       (time) datetime64[ns] 2019-01-01 ... 2025-01-22T00:15:00
  * y          (y) int32 0 1 2 3 4 5 6 7 ... 2809 2810 2811 2812 2813 2814 2815
  * x          (x) int32 0 1 2 3 4 5 6 7 ... 2798 2799 2800 2801 2802 2803 2804
Dimensions without coordinates: dim_0, dim_1, dim_2
Data variables:
    Latitude   (time, y, x) float64 ...
    Longitude  (time, y, x) float64 ...
    SST_WKL    (time, dim_0, dim_1, dim_2) float32 ...
Attributes:
    title:         Combined MOSDAC SST Data (Dask-processed)
    source_file:   Week-1-APR-2019-0000.h5
    created:       2025-03-28T11:41:53.210311
    source_files:  ['Week-1-APR-2019-0000.nc', 'Week-1-APR-2020-0000.nc', 'We...

######  remove dimensions 'dim_0', 'dim_1', 'dim_2' and variables GeoY , GeoX , BINNING_COUNTER, SST_QFLAGS_WKL from dataset

In [5]:
def clean_netcdf_file(
    input_file,
    variables_to_remove=None,
    dimensions_to_remove=None,
    variables_to_preserve=None,  
    create_backup=True,
    chunk_size={"time": 30, "x": 500, "y": 500}
):
    """
    Removes specified variables and dimensions from a NetCDF file efficiently.

    Parameters:
    -----------
    input_file : str
        Path to the NetCDF file to modify
    variables_to_remove : list, optional
        List of variable names to remove from the NetCDF file
    dimensions_to_remove : list, optional
        List of dimension names to remove from the NetCDF file (dim_0, dim_1, etc.)
    variables_to_preserve : list, optional
        List of variable names to always preserve, even if they use dimensions to be removed
    create_backup : bool, optional
        Whether to create a backup of the original file before modifying (default: True)
    chunk_size : dict, optional
        Dictionary specifying chunk sizes for dimensions
    """
    # Initialize empty lists if None is provided
    if variables_to_remove is None:
        variables_to_remove = []
    if dimensions_to_remove is None:
        dimensions_to_remove = []
    if variables_to_preserve is None:
        variables_to_preserve = []
       
    # Create backup 
    backup_file = input_file + ".backup"
    if create_backup and not os.path.exists(backup_file):
        print(f"Creating backup at {backup_file}")
        copyfile(input_file, backup_file)

    try:
        # First, open the dataset with minimal loading to check structure
        with xr.open_dataset(input_file, decode_cf=False) as ds_small:
            # Get the actual dimensions in the file for efficient chunking
            actual_dims = ds_small.dims
           
            # Create a chunking strategy that only includes dimensions that exist
            effective_chunks = {dim: size for dim, size in chunk_size.items()
                              if dim in actual_dims}
           
            # Check which variables and dimensions actually exist
            existing_vars = [var for var in variables_to_remove if var in ds_small.variables]
            existing_dims = [dim for dim in dimensions_to_remove if dim in ds_small.dims]
           
            # Ensure variables_to_preserve exist in the dataset
            existing_preserve = [var for var in variables_to_preserve if var in ds_small.variables]
           
            # Find variables that use dimensions to be removed (but are not in the preserve list)
            vars_using_dims = []
            if existing_dims:
                for var_name, var in ds_small.variables.items():
                    if any(dim in var.dims for dim in existing_dims) and var_name not in existing_preserve:
                        vars_using_dims.append(var_name)
           
            # Determine which variables we'll actually keep
            all_vars = list(ds_small.variables)
            vars_to_keep = [var for var in all_vars
                          if (var not in existing_vars and var not in vars_using_dims) or var in existing_preserve]
       
        # Now reopen with proper chunking and only load the variables we need
        with xr.open_dataset(input_file, chunks=effective_chunks, decode_cf=False) as ds:
            # Create a new dataset with only the variables we want to keep
            ds_reduced = ds[vars_to_keep]
           
            # Report what we're doing
            if existing_vars:
                print(f"Removing variables: {[v for v in existing_vars if v not in existing_preserve]}")
            if vars_using_dims:
                print(f"Removing variables that use dimensions {existing_dims}: {vars_using_dims}")
            if existing_preserve:
                print(f"Preserving essential variables: {existing_preserve}")
            if not (existing_vars or vars_using_dims):
                print("No variables to remove.")
                return True  # Nothing to do
               
            # Save directly to a temporary file with compression
            temp_file = input_file + ".temp"
            delayed_obj = ds_reduced.to_netcdf(
                temp_file,
                engine="netcdf4",
                compute=False,
                encoding={var: {'zlib': True, 'complevel': 1} for var in vars_to_keep}
            )
           
            print(f"Processing and writing {input_file}...")
            dask.compute(delayed_obj)
           
            # Force Python garbage collection
            ds_reduced = None
            gc.collect()
       
        # Replace the original file
        os.remove(input_file)
        os.rename(temp_file, input_file)
        print(f"Successfully updated {input_file}")
        return True
       
    except Exception as e:
        print(f"Error processing {input_file}: {str(e)}")
        # Clean up temp file if it exists
        temp_file = input_file + ".temp"
        if os.path.exists(temp_file):
            os.remove(temp_file)
        return False



if __name__ == "__main__":
    
    input_file = r"C:\Users\Admin\RIYA PROJECT\DATASETS\mosdac_sst_cleaned.nc"
    variables_to_remove = ["GeoY", "GeoX", "BINNING_COUNTER", "SST_QFLAGS_WKL"]
    dimensions_to_remove = ["dim_0", "dim_1", "dim_2"] 
    variables_to_preserve = ["SST_WKL","Longitude","Latitude"] 
   
    success = clean_netcdf_file(
        input_file,
        variables_to_remove,
        dimensions_to_remove,
        variables_to_preserve,
        chunk_size={"time": 30, "lat": 20, "lon": 20}  # Adjust based on your data
    )

    if success:
        print("File was successfully cleaned.")
   
    

Removing variables: ['GeoY', 'GeoX', 'BINNING_COUNTER', 'SST_QFLAGS_WKL']
Removing variables that use dimensions ['dim_0', 'dim_1', 'dim_2']: ['BINNING_COUNTER', 'SST_QFLAGS_WKL']
Preserving essential variables: ['SST_WKL', 'Longitude', 'Latitude']
Processing and writing C:\Users\Admin\RIYA PROJECT\DATASETS\mosdac_sst_cleaned.nc...
Successfully updated C:\Users\Admin\RIYA PROJECT\DATASETS\mosdac_sst_cleaned.nc
File was successfully cleaned.


In [10]:
#CHECKING WHY DIMENSIONS WEREN'T REMOVED

def clean_netcdf_file(
    input_file,
    variables_to_remove=None,
    dimensions_to_remove=None,
    variables_to_preserve=None,  
    create_backup=True,
    chunk_size={"time": 30, "x": 500, "y": 500}
):
    """
    Removes specified variables and dimensions from a NetCDF file efficiently.

    Parameters:
    -----------
    input_file : str
        Path to the NetCDF file to modify
    variables_to_remove : list, optional
        List of variable names to remove from the NetCDF file
    dimensions_to_remove : list, optional
        List of dimension names to remove from the NetCDF file (dim_0, dim_1, etc.)
    variables_to_preserve : list, optional
        List of variable names to always preserve, even if they use dimensions to be removed
    create_backup : bool, optional
        Whether to create a backup of the original file before modifying (default: True)
    chunk_size : dict, optional
        Dictionary specifying chunk sizes for dimensions
    """
    # Initialize empty lists if None is provided
    if variables_to_remove is None:
        variables_to_remove = []
    if dimensions_to_remove is None:
        dimensions_to_remove = []
    if variables_to_preserve is None:
        variables_to_preserve = []
       
    # Create backup if requested - do this only once
    backup_file = input_file + ".backup"
    if create_backup and not os.path.exists(backup_file):
        print(f"Creating backup at {backup_file}")
        copyfile(input_file, backup_file)

    try:
        # First, open the dataset with minimal loading to check structure
        with xr.open_dataset(input_file, decode_cf=False) as ds_small:
            # Get the actual dimensions in the file for efficient chunking
            actual_dims = ds_small.dims
           
            # Create a chunking strategy that only includes dimensions that exist
            effective_chunks = {dim: size for dim, size in chunk_size.items()
                              if dim in actual_dims}
           
            # Check which variables and dimensions actually exist
            existing_vars = [var for var in variables_to_remove if var in ds_small.variables]
            existing_dims = [dim for dim in dimensions_to_remove if dim in ds_small.dims]
           
            # Ensure variables_to_preserve exist in the dataset
            existing_preserve = [var for var in variables_to_preserve if var in ds_small.variables]
           
            # Find variables that use dimensions to be removed
            vars_using_dims = []
            preserve_uses_dims_to_remove = False
            dims_used_by_preserve = []
           
            if existing_dims:
                for var_name, var in ds_small.variables.items():
                    dims_used = [dim for dim in var.dims if dim in existing_dims]
                    if dims_used:
                        if var_name in existing_preserve:
                            preserve_uses_dims_to_remove = True
                            dims_used_by_preserve.extend(dims_used)
                        else:
                            vars_using_dims.append(var_name)
           
            # Get unique list of dimensions that must be kept
            dims_used_by_preserve = list(set(dims_used_by_preserve))
           
            # Update dimensions to remove, excluding any used by preserved variables
            effective_dims_to_remove = [dim for dim in existing_dims if dim not in dims_used_by_preserve]
           
            # Re-evaluate variables that use dimensions to be effectively removed
            if effective_dims_to_remove != existing_dims:
                vars_using_dims = []
                for var_name, var in ds_small.variables.items():
                    if any(dim in var.dims for dim in effective_dims_to_remove) and var_name not in existing_preserve:
                        vars_using_dims.append(var_name)
           
            # Determine which variables we'll actually keep
            all_vars = list(ds_small.variables)
            vars_to_keep = [var for var in all_vars
                          if (var not in existing_vars and var not in vars_using_dims) or var in existing_preserve]
       
        # Now reopen with proper chunking and only load the variables we need
        with xr.open_dataset(input_file, chunks=effective_chunks, decode_cf=False) as ds:
            # Create a new dataset with only the variables we want to keep
            ds_reduced = ds[vars_to_keep]
           
            # Report what we're doing
            if existing_vars:
                print(f"Removing variables: {[v for v in existing_vars if v not in existing_preserve]}")
           
            if preserve_uses_dims_to_remove:
                print(f"WARNING: Preserved variables {existing_preserve} use dimensions {dims_used_by_preserve}")
                print(f"These dimensions will NOT be removed to maintain data integrity")
                print(f"Only removing dimensions: {effective_dims_to_remove}")
           
            if vars_using_dims:
                print(f"Removing variables that use dimensions {effective_dims_to_remove}: {vars_using_dims}")
           
            if existing_preserve:
                print(f"Preserving essential variables: {existing_preserve}")
               
            if not (existing_vars or vars_using_dims):
                print("No variables to remove.")
                return True  # Nothing to do
               
            # Save directly to a temporary file with compression
            temp_file = input_file + ".temp"
            delayed_obj = ds_reduced.to_netcdf(
                temp_file,
                engine="netcdf4",
                compute=False,
                encoding={var: {'zlib': True, 'complevel': 1} for var in vars_to_keep}
            )
           
            print(f"Processing and writing {input_file}...")
            dask.compute(delayed_obj)
           
            # Force Python garbage collection
            ds_reduced = None
            gc.collect()
       
        # Replace the original file
        os.remove(input_file)
        os.rename(temp_file, input_file)
        print(f"Successfully updated {input_file}")
        return True
       
    except Exception as e:
        print(f"Error processing {input_file}: {str(e)}")
        # Clean up temp file if it exists
        temp_file = input_file + ".temp"
        if os.path.exists(temp_file):
            os.remove(temp_file)
        return False



if __name__ == "__main__":

    input_file = r"C:\Users\Admin\RIYA PROJECT\DATASETS\mosdac_sst_cleaned.nc"
    variables_to_remove = ["GeoY", "GeoX", "BINNING_COUNTER", "SST_QFLAGS_WKL"]
    dimensions_to_remove = ["dim_0", "dim_1", "dim_2"]  # Add dimension names you want to remove
    variables_to_preserve = ["SST_WKL","Latitude","Longitude"]  # Always keep this variable
   
    # Clean a single file
    success = clean_netcdf_file(
        input_file,
        variables_to_remove,
        dimensions_to_remove,
        variables_to_preserve,
        chunk_size={"time": 30, "x": 500, "y": 500}  # Adjust based on your data
    )

    if success:
        print("File was successfully cleaned.")   

These dimensions will NOT be removed to maintain data integrity
Only removing dimensions: []
Preserving essential variables: ['SST_WKL', 'Latitude', 'Longitude']
No variables to remove.
File was successfully cleaned.


The dimensions cant be removed because the variables are using them 

In [9]:
f = xr.open_dataset(r"C:\Users\Admin\RIYA PROJECT\DATASETS\mosdac_sst_cleaned.nc")
f

<xarray.Dataset>
Dimensions:    (time: 278, y: 2816, x: 2805, dim_0: 1, dim_1: 2816, dim_2: 2805)
Coordinates:
  * time       (time) datetime64[ns] 2019-01-01 ... 2025-01-01T00:15:00
  * y          (y) int32 0 1 2 3 4 5 6 7 ... 2809 2810 2811 2812 2813 2814 2815
  * x          (x) int32 0 1 2 3 4 5 6 7 ... 2798 2799 2800 2801 2802 2803 2804
Dimensions without coordinates: dim_0, dim_1, dim_2
Data variables:
    Latitude   (time, y, x) int16 ...
    Longitude  (time, y, x) int16 ...
    SST_WKL    (time, dim_0, dim_1, dim_2) float32 ...
Attributes:
    title:              Combined MOSDAC SST Data (Dask-optimized)
    source_file:        Week-1-JAN-2019-0000.h5
    created:            2025-04-01T11:38:50.536703
    source_file_count:  278

###### MAKING MONTHLY MEAN FROM WEEKLY MEAN

In [9]:
def process_mosdac_sst(input_dataset):
    """
    Process MOSDAC INSAT 3DR SST dataset to create monthly means   
    Parameters:
    input_dataset : xarray.Dataset
        Original MOSDAC INSAT 3DR SST dataset   
    Returns:
    xarray.Dataset
        Monthly mean dataset with standardized coordinates
    """
    # Rename dimensions and coordinates to match other datasets
    renamed_dataset = input_dataset.rename({
        'time': 'time',
        'Latitude': 'lat',
        'Longitude': 'lon',
        'SST_WKL': 'sst'
    })
   
    # Remove additional dimensions if they're not needed
    renamed_dataset = renamed_dataset.drop_vars(['dim_0', 'dim_1', 'dim_2'], errors='ignore')
   
    # Ensure time coordinate is datetime
    renamed_dataset['time'] = renamed_dataset['time'].astype('datetime64[ns]')
   
    # Calculate monthly means
    monthly_mean = renamed_dataset.groupby('time.month').mean()
   
    # Rename month coordinate
    monthly_mean = monthly_mean.rename({'month': 'time'})
   
    return monthly_mean


ds = xr.open_dataset(r"C:\Users\Admin\RIYA PROJECT\DATASETS\mosdac_sst_cleaned.nc")
monthly_mean_ds = process_mosdac_sst(ds)
monthly_mean_ds.to_netcdf(r"C:\Users\Admin\RIYA PROJECT\DATASETS\mosdac_mon_mean.nc")

print("MOSDAC SST monthly mean processing function created successfully.")
print("Key features:")
print("- Renamed coordinates to match standard conventions (lat, lon, time)")
print("- Removes additional dimensions")
print("- Calculates monthly means")
print("- Aligns with other datasets' naming")

MOSDAC SST monthly mean processing function created successfully.
Key features:
- Renamed coordinates to match standard conventions (lat, lon, time)
- Removes additional dimensions
- Calculates monthly means
- Aligns with other datasets' naming


In [10]:
sst = xr.open_dataset(r"C:\Users\Admin\RIYA PROJECT\DATASETS\mosdac_mon_mean.nc")
sst

<xarray.Dataset>
Dimensions:  (y: 2816, x: 2805, time: 12, dim_0: 1, dim_1: 2816, dim_2: 2805)
Coordinates:
  * y        (y) int32 0 1 2 3 4 5 6 7 ... 2809 2810 2811 2812 2813 2814 2815
  * x        (x) int32 0 1 2 3 4 5 6 7 ... 2798 2799 2800 2801 2802 2803 2804
  * time     (time) int64 1 2 3 4 5 6 7 8 9 10 11 12
Dimensions without coordinates: dim_0, dim_1, dim_2
Data variables:
    lat      (time, y, x) float64 ...
    lon      (time, y, x) float64 ...
    sst      (time, dim_0, dim_1, dim_2) float32 ...

In [11]:
# CHECKING DATA QUALITY

def assess_netcdf_quality(netcdf_file):
    """
    Perform comprehensive quality assessment on a NetCDF file.

    Parameters:
    -----------
    netcdf_file : str
        Path to the NetCDF file to assess
    """
    print(f"Loading NetCDF file: {netcdf_file}")
    ds = xr.open_dataset(netcdf_file)

    # 1. Basic Dataset Information
    print("\n=== BASIC DATASET INFORMATION ===")
    print(f"Dimensions: {ds.dims}")
    print(f"Variables: {list(ds.variables)}")
    print(f"Attributes: {ds.attrs}")

    # 2. Check for Time Coverage (if time dimension exists)
    if 'time' in ds.dims or 'time' in ds.coords:
        time_var = 'time'
    elif any(dim for dim in ds.dims if 'time' in dim.lower()):
        time_var = next(dim for dim in ds.dims if 'time' in dim.lower())
    else:
        time_var = None

    if time_var:
        print("\n=== TIME COVERAGE ===")
        time_data = ds[time_var]
        print(f"Start time: {time_data.values[0]}")
        print(f"End time: {time_data.values[-1]}")
        print(f"Number of time steps: {len(time_data)}")

        # Check for gaps in time series
        if len(time_data) > 1:
            time_diff = np.diff(time_data.values)
            unique_diffs = np.unique(time_diff)
            if len(unique_diffs) > 1:
                print("WARNING: Irregular time intervals detected!")
                print(f"Time differences: {unique_diffs}")

    # 3. Data Variable Analysis
    print("\n=== DATA VARIABLE ANALYSIS ===")
    data_vars = list(ds.data_vars)

    for var_name in data_vars:
        var = ds[var_name]
        print(f"\nAnalyzing variable: {var_name}")
        print(f"  - Shape: {var.shape}")
        print(f"  - Data type: {var.dtype}")

        if hasattr(var, 'units'):
            print(f"  - Units: {var.units}")
        elif 'units' in var.attrs:
            print(f"  - Units: {var.attrs['units']}")

        # Calculate basic statistics
        if np.issubdtype(var.dtype, np.number):
            with np.errstate(invalid='ignore'):
                stats_dict = {
                    'Min': float(var.min().values),
                    'Max': float(var.max().values),
                    'Mean': float(var.mean().values),
                    'Median': float(np.nanmedian(var.values)),
                    'Std Dev': float(var.std().values),
                    'Missing values (%)': float(np.isnan(var).sum().values / var.size * 100)
                }

                print("  - Statistics:")
                for stat_name, stat_value in stats_dict.items():
                    print(f"    * {stat_name}: {stat_value}")

                # Check for outliers using IQR method
                q1 = float(np.nanpercentile(var.values, 25))
                q3 = float(np.nanpercentile(var.values, 75))
                iqr = q3 - q1
                lower_bound = q1 - 1.5 * iqr
                upper_bound = q3 + 1.5 * iqr
                outliers = ((var < lower_bound) | (var > upper_bound)).sum().values
                outlier_percentage = float(outliers / var.size * 100)

                print(f"  - Outliers: {outliers} ({outlier_percentage:.2f}% of data)")

    # 4. Data Visualization
    print("\n=== GENERATING VISUALIZATIONS ===")

    # Select numerical variables for visualization
    numerical_vars = [var for var in data_vars
                     if np.issubdtype(ds[var].dtype, np.number)
                     and ds[var].size > 0
                     and ds[var].ndim <= 2]  # Limit to 2D for simplicity

    if numerical_vars:
        # 4.1 Histograms for distribution
        plt.figure(figsize=(15, 10))
        for i, var_name in enumerate(numerical_vars[:min(6, len(numerical_vars))]):  # Limit to 6 variables
            plt.subplot(2, 3, i+1)
            da = ds[var_name]

            # Flatten for histogram
            flat_data = da.values.flatten()
            valid_data = flat_data[~np.isnan(flat_data)]

            if len(valid_data) > 0:
                plt.hist(valid_data, bins=30, alpha=0.7)
                plt.title(f'Distribution of {var_name}')
                plt.xlabel(var_name)
                plt.ylabel('Frequency')

        plt.tight_layout()

        # 4.2 Correlation matrix if multiple variables
        if len(numerical_vars) > 1:
            # Create dataframe for correlation analysis
            data_dict = {}
            common_size = None

            # Find variables that can be flattened to the same length
            for var_name in numerical_vars:
                flat_data = ds[var_name].values.flatten()
                if common_size is None:
                    common_size = len(flat_data)

                if len(flat_data) == common_size:
                    data_dict[var_name] = flat_data

            if len(data_dict) > 1:
                df = pd.DataFrame(data_dict)

                # Calculate correlation matrix
                plt.figure(figsize=(10, 8))
                corr_matrix = df.corr(method='pearson')
                sns.heatmap(corr_matrix, annot=True, cmap='coolwarm', vmin=-1, vmax=1)
                plt.title('Correlation Matrix')
                plt.tight_layout()

        # 4.3 Time series plot if time dimension exists
        if time_var and len(numerical_vars) > 0:
            for var_name in numerical_vars[:min(3, len(numerical_vars))]:
                var = ds[var_name]

                # Check if we can create a time series
                if time_var in var.dims:
                    plt.figure(figsize=(12, 6))

                    # If it's multidimensional, take mean across other dimensions
                    if var.ndim > 1:
                        dims_to_mean = [dim for dim in var.dims if dim != time_var]
                        if dims_to_mean:
                            var_plot = var.mean(dim=dims_to_mean)
                        else:
                            var_plot = var
                    else:
                        var_plot = var

                    var_plot.plot()
                    plt.title(f'Time Series of {var_name}')
                    plt.tight_layout()


    # 5. Check Spatial Coverage (if applicable)
    spatial_dims = [dim for dim in ds.dims if dim.lower() in ['lat', 'latitude', 'lon', 'longitude', 'x', 'y']]
    if len(spatial_dims) >= 2:
        print("\n=== SPATIAL COVERAGE ===")
        for dim in spatial_dims:
            if dim in ds.coords:
                print(f"{dim}: {ds.coords[dim].values.min()} to {ds.coords[dim].values.max()}")

    # 6. Check for Suspicious Values
    print("\n=== CHECKING FOR SUSPICIOUS VALUES ===")
    for var_name in data_vars:
        var = ds[var_name]
        if np.issubdtype(var.dtype, np.number):
            # Check for excessive zeros
            zero_percentage = float((var == 0).sum().values / var.size * 100)
            if zero_percentage > 30:  # Arbitrary threshold
                print(f"WARNING: {var_name} has {zero_percentage:.2f}% zeros")

            # Check for unrealistic constant values
            unique_vals = np.unique(var.values[~np.isnan(var.values)])
            if len(unique_vals) == 1:
                print(f"WARNING: {var_name} has only one unique value: {unique_vals[0]}")
            elif len(unique_vals) < 10 and var.size > 1000:
                print(f"WARNING: {var_name} has suspiciously few unique values ({len(unique_vals)})")

    # 7. Overall Data Quality Score
    print("\n=== OVERALL DATA QUALITY ASSESSMENT ===")
    quality_issues = []

    # Check missing data percentage across all variables
    total_missing = sum(np.isnan(ds[var]).sum().values for var in data_vars if np.issubdtype(ds[var].dtype, np.number))
    total_points = sum(ds[var].size for var in data_vars if np.issubdtype(ds[var].dtype, np.number))
    missing_percentage = (total_missing / total_points * 100) if total_points > 0 else 0

    if missing_percentage > 10:
        quality_issues.append(f"High missing data percentage: {missing_percentage:.2f}%")

    # Final quality verdict
    if len(quality_issues) == 0:
        print("No major quality issues detected.")
        print("Overall data quality appears to be good.")
    else:
        print("Quality issues detected:")
        for issue in quality_issues:
            print(f"- {issue}")

        if len(quality_issues) <= 2:
            print("Overall data quality appears to be moderate with some issues.")
        else:
            print("Overall data quality appears to be poor with multiple issues.")

    # Close the dataset
    ds.close()
    print("\nQuality assessment complete.")

# Example usage
if __name__ == "__main__":
    netcdf_file = r"C:\Users\Admin\RIYA PROJECT\DATASETS\mosdac_mon_mean.nc"
    assess_netcdf_quality(netcdf_file)

Loading NetCDF file: C:\Users\Admin\RIYA PROJECT\DATASETS\mosdac_mon_mean.nc

=== BASIC DATASET INFORMATION ===
Dimensions: Frozen({'y': 2816, 'x': 2805, 'time': 12, 'dim_0': 1, 'dim_1': 2816, 'dim_2': 2805})
Variables: ['y', 'x', 'lat', 'lon', 'sst', 'time']
Attributes: {}

=== TIME COVERAGE ===
Start time: 1
End time: 12
Number of time steps: 12

=== DATA VARIABLE ANALYSIS ===

Analyzing variable: lat
  - Shape: (12, 2816, 2805)
  - Data type: float64
  - Statistics:
    * Min: -32768.0
    * Max: 32767.0
    * Mean: 8864.260166884449
    * Median: 1522.0
    * Std Dev: 14772.995276299169
    * Missing values (%): 0.0
  - Outliers: 0 (0.00% of data)

Analyzing variable: lon
  - Shape: (12, 2816, 2805)
  - Data type: float64
  - Statistics:
    * Min: -715.0
    * Max: 32767.0
    * Mean: 14264.021585677696
    * Median: 9197.0
    * Std Dev: 11622.541974279327
    * Missing values (%): 0.0
  - Outliers: 0 (0.00% of data)

Analyzing variable: sst
  - Shape: (12, 1, 2816, 2805)
  - Dat

In [42]:
# # ADRESSING QUALITY ISSUES LIKE : Handling missing values, Normalizing longitude values, Temperature unit verification, Quality assessment summary, Temporal pattern analysis

# def load_and_clean_sst_data(file_path):
#     """
#     Load and clean MOSDAC SST NetCDF data.

#     Parameters:
#     file_path (str): Path to the NetCDF file

#     Returns:
#     xarray.Dataset: Cleaned dataset
#     """
#     print(f"Loading NetCDF file: {file_path}")
#     ds = xr.open_dataset(file_path)

#     # Print basic information about the dataset
#     print("=== ORIGINAL DATASET INFO ===")
#     print(f"Dimensions: {ds.dims}")
#     print(f"Variables: {list(ds.variables)}")

#     # Handle missing values in SST data
#     # Replace -999.0 with NaN
#     if 'SST_WKL' in ds:
#         print("\n=== HANDLING MISSING VALUES ===")
#         # Get stats before cleaning
#         valid_before = (~np.isclose(ds.SST_WKL, -999.0)).sum().item()
#         total_points = ds.SST_WKL.size
#         print(f"Before cleaning: {valid_before}/{total_points} valid points ({valid_before/total_points*100:.2f}%)")

#         # Replace fill values with NaN
#         ds['SST_WKL'] = ds.SST_WKL.where(~np.isclose(ds.SST_WKL, -999.0))

#         # Get stats after cleaning
#         valid_after = (~np.isnan(ds.SST_WKL)).sum().values
#         print(f"After cleaning: {valid_after}/{total_points} valid points ({valid_after/total_points*100:.2f}%)")

#     # Fix longitude range if needed
#     if 'Longitude' in ds:
#         print("\n=== NORMALIZING LONGITUDE VALUES ===")
#         # Check current range
#         lon_min = ds.Longitude.min().values
#         lon_max = ds.Longitude.max().values
#         print(f"Original longitude range: {lon_min} to {lon_max}")

#         # Normalize to -180 to 180 range
#         if lon_min < -180 or lon_max > 180:
#             # Apply modulo 360 and adjust to -180 to 180 range
#             ds['Longitude'] = ((ds.Longitude + 180) % 360) - 180
#             print(f"Normalized longitude range: {ds.Longitude.min().values} to {ds.Longitude.max().values}")

#     # Check temperature values and units
#     if 'SST_WKL' in ds:
#         print("\n=== CHECKING TEMPERATURE VALUES ===")
#         valid_temp = ds.SST_WKL.where(~np.isnan(ds.SST_WKL))
#         temp_min = valid_temp.min().values
#         temp_max = valid_temp.max().values
#         temp_mean = valid_temp.mean().values

#         print(f"Temperature range: {temp_min} to {temp_max}")
#         print(f"Temperature mean: {temp_mean}")

#         # Check if temperature appears to be in Kelvin (typically 273-313K)
#         if temp_max > 200:
#             print("Temperature appears to be in Kelvin.")
#             # Convert to Celsius if needed
#             ds['SST_WKL_C'] = ds.SST_WKL - 273.15
#             print(f"Created Celsius version: {ds.SST_WKL_C.min().values} to {ds.SST_WKL_C.max().values}")
#         else:
#             print("Temperature appears to be in Celsius.")

#     # Create a quality assessment summary
#     missing_pct = np.isnan(ds.SST_WKL).sum().values / ds.SST_WKL.size * 100
#     print(f"\n=== QUALITY ASSESSMENT SUMMARY ===")
#     print(f"Total data points: {ds.SST_WKL.size}")
#     print(f"Missing data points: {np.isnan(ds.SST_WKL).sum().values} ({missing_pct:.2f}%)")

#     if missing_pct > 50:
#         print("WARNING: More than 50% of data points are missing. Consider using a different dataset.")

#     return ds

# # Function to extract and summarize spatial-temporal patterns
# def analyze_temporal_patterns(ds):
#     """
#     Analyze temporal patterns in the SST data

#     Parameters:
#     ds (xarray.Dataset): Cleaned dataset

#     Returns:
#     dict: Summary statistics
#     """
#     # Ensure we're working with the temperature in Celsius
#     temp_var = 'SST_WKL_C' if 'SST_WKL_C' in ds else 'SST_WKL'

#     print("\n=== MONTHLY PATTERNS ===")
   
#     # Direct calculation for each month separately using the existing 'month' dimension
#     # This works because 'month' is already a dimension in the dataset structure
#     for m in range(1, 13):
#         # Select data for this month
#         month_data = ds[temp_var].sel(month=m)
       
#         # Calculate statistics, explicitly specifying dimensions to reduce over
#         mean_val = month_data.mean(dim=['dim_0', 'dim_1', 'dim_2'], skipna=True).item()
#         # For std, check if there's enough non-NaN data to calculate a meaningful std
#         if (~np.isnan(month_data)).sum() > 1:  # Need at least 2 points for std
#             std_val = month_data.std(dim=['dim_0', 'dim_1', 'dim_2'], skipna=True).item()
#         else:
#             std_val = np.nan
       
#         # Check if we got valid statistics before printing
#         if not np.isnan(mean_val):
#             print(f"Month {m}: Mean = {mean_val:.2f}, " +
#                   (f"Std = {std_val:.2f}" if not np.isnan(std_val) else "Std = N/A"))
#         else:
#             print(f"Month {m}: No valid data available")

#     # Create summary stats
#     summary = {
#         'spatial_coverage': {
#             'latitude_range': [float(ds.Latitude.min().values), float(ds.Latitude.max().values)],
#             'longitude_range': [float(ds.Longitude.min().values), float(ds.Longitude.max().values)]
#         },
#         'temporal_coverage': {
#             'months': list(np.unique(ds.month.values)) if 'month' in ds else []
#         },
#         'data_quality': {
#             'missing_percentage': float(np.isnan(ds[temp_var]).sum().values / ds[temp_var].size * 100),
#             'temperature_range': [float(ds[temp_var].min().values), float(ds[temp_var].max().values)]
#         }
#     }

#     return summary

# # SAVE CHANGES TO FILE
# def save_cleaned_dataset(ds, output_path):
#     """
#     Save the cleaned dataset to a new NetCDF file

#     Parameters:
#     ds (xarray.Dataset): Cleaned dataset
#     output_path (str): Path where the cleaned file will be saved
#     """
#     print(f"\n=== SAVING CLEANED DATASET ===")
#     print(f"Saving to: {output_path}")

#     # Add attributes to document the cleaning process
#     ds.attrs['cleaning_date'] = str(datetime.now())
#     ds.attrs['cleaning_steps'] = "Removed -999 fill values; Normalized longitude to -180/180 range; Filtered physically impossible SST values; Added Celsius conversion"

#     # Save to NetCDF file
#     ds.to_netcdf(output_path)
#     print(f"Dataset successfully saved to {output_path}")
#     return True


# if __name__ == "__main__":
#     # Replace with your actual file paths
#     input_file_path = r"C:\Users\Admin\RIYA PROJECT\DATASETS\mosdac_konkan_sst.nc"
#     output_file_path = r"C:\Users\Admin\RIYA PROJECT\DATASETS\mosdac_konkan_sst_cleaned.nc"

#     # Load and clean the data
#     clean_ds = load_and_clean_sst_data(input_file_path)

#     # Add additional filtering for physically impossible  SST values
#     if 'SST_WKL' in clean_ds:
#         print("\n=== FILTERING PHYSICALLY IMPOSSIBLE VALUES ===")

#         # Count before filtering
#         valid_before = (~np.isnan(clean_ds.SST_WKL)).sum().values

#         # Apply physical constraints (SST range: -2°C to 35°C in Kelvin)
#         clean_ds['SST_WKL'] = clean_ds.SST_WKL.where((clean_ds.SST_WKL >= 271) & (clean_ds.SST_WKL <= 308))

#         # Count after filtering
#         valid_after = (~np.isnan(clean_ds.SST_WKL)).sum().values

#         print(f"Before filtering: {valid_before} valid points")
#         print(f"After filtering: {valid_after} valid points")
#         print(f"Removed {valid_before - valid_after} physically impossible values")

#         # Also update the Celsius version
#         if 'SST_WKL_C' in clean_ds:
#             clean_ds['SST_WKL_C'] = clean_ds.SST_WKL - 273.15

#     # Analyze temporal patterns
#     summary = analyze_temporal_patterns(clean_ds)

#     # Save the cleaned dataset
#     save_cleaned_dataset(clean_ds, output_file_path)

#     print("\n=== DATA READY FOR ANALYSIS ===")
#     print("Cleaned dataset has been saved and is ready for further analysis.")

Loading NetCDF file: C:\Users\Admin\RIYA PROJECT\DATASETS\mosdac_konkan_sst.nc
=== ORIGINAL DATASET INFO ===
Dimensions: Frozen({'y': 16, 'x': 21, 'month': 12, 'dim_0': 1, 'dim_1': 2816, 'dim_2': 2805})
Variables: ['y', 'x', 'Latitude', 'Longitude', 'SST_WKL', 'month', 'month_name']

=== HANDLING MISSING VALUES ===
Before cleaning: 32395295/94786560 valid points (34.18%)
After cleaning: 32395295/94786560 valid points (34.18%)

=== NORMALIZING LONGITUDE VALUES ===
Original longitude range: -306.0 to 326.0
Normalized longitude range: -177.0 to 159.0

=== CHECKING TEMPERATURE VALUES ===
Temperature range: -959.040771484375 to 307.83404541015625
Temperature mean: 211.04525756835938
Temperature appears to be in Kelvin.
Created Celsius version: -1232.1907958984375 to 34.684051513671875

=== QUALITY ASSESSMENT SUMMARY ===
Total data points: 94786560
Missing data points: 62391265 (65.82%)

=== FILTERING PHYSICALLY IMPOSSIBLE VALUES ===
Before filtering: 32395295 valid points
After filtering: 2

# ***COMPARING WITH NOAA SST DATA***

In [7]:
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from datetime import datetime

def load_dataset(file_path, dataset_name):
    """Load and provide basic info about a NetCDF dataset"""
    print(f"\n=== LOADING {dataset_name} DATASET ===")
    try:
        ds = xr.open_dataset(file_path)
        print(f"Successfully loaded: {file_path}")
        print(f"Dimensions: {ds.dims}")
        print(f"Variables: {list(ds.variables)}")

        # Find temperature variable (could be SST, sst, sea_surface_temperature, etc.)
        temp_vars = [var for var in ds.variables if 'sst' in var.lower() or 'temp' in var.lower()]
        if temp_vars:
            print(f"Found temperature variables: {temp_vars}")

            # Get basic stats for the first temperature variable
            temp_var = temp_vars[0]
            valid_data = ds[temp_var].where(~np.isnan(ds[temp_var]))
            print(f"Temperature range: {valid_data.min().values} to {valid_data.max().values}")
            print(f"Missing values: {np.isnan(ds[temp_var]).sum().values/ds[temp_var].size*100:.2f}%")
        else:
            print("No obvious temperature variable found")

        return ds
    except Exception as e:
        print(f"Error loading {file_path}: {e}")
        return None

def compare_datasets(mosdac_ds, noaa_ds):
    """Compare two SST datasets"""
    print("\n=== COMPARING DATASETS ===")

    # Identify temperature variables
    mosdac_temp_var = 'SST_WKL' if 'SST_WKL' in mosdac_ds else 'SST_WKL_C'

    # Try to identify NOAA temperature variable
    noaa_temp_vars = [var for var in noaa_ds.variables if 'sst' in var.lower() or 'temp' in var.lower()]
    if not noaa_temp_vars:
        print("Could not identify temperature variable in NOAA dataset")
        return

    noaa_temp_var = noaa_temp_vars[0]
    print(f"Comparing {mosdac_temp_var} (MOSDAC) with {noaa_temp_var} (NOAA)")

    # 1. Compare spatial coverage
    print("\n--- Spatial Coverage Comparison ---")
    try:
        mosdac_lat_range = [float(mosdac_ds.Latitude.min().values), float(mosdac_ds.Latitude.max().values)]
        mosdac_lon_range = [float(mosdac_ds.Longitude.min().values), float(mosdac_ds.Longitude.max().values)]

        # Find lat/lon variables in NOAA dataset
        noaa_lat_var = [var for var in noaa_ds.variables if 'lat' in var.lower()][0]
        noaa_lon_var = [var for var in noaa_ds.variables if 'lon' in var.lower()][0]

        noaa_lat_range = [float(noaa_ds[noaa_lat_var].min().values), float(noaa_ds[noaa_lat_var].max().values)]
        noaa_lon_range = [float(noaa_ds[noaa_lon_var].min().values), float(noaa_ds[noaa_lon_var].max().values)]

        print(f"MOSDAC latitude range: {mosdac_lat_range}")
        print(f"NOAA latitude range: {noaa_lat_range}")
        print(f"MOSDAC longitude range: {mosdac_lon_range}")
        print(f"NOAA longitude range: {noaa_lon_range}")
    except Exception as e:
        print(f"Error comparing spatial coverage: {e}")

    

    # 3. Compare data completeness
    print("\n--- Data Completeness Comparison ---")
    try:
        mosdac_missing = np.isnan(mosdac_ds[mosdac_temp_var]).sum().values / mosdac_ds[mosdac_temp_var].size * 100
        noaa_missing = np.isnan(noaa_ds[noaa_temp_var]).sum().values / noaa_ds[noaa_temp_var].size * 100

        print(f"MOSDAC missing data: {mosdac_missing:.2f}%")
        print(f"NOAA missing data: {noaa_missing:.2f}%")
    except Exception as e:
        print(f"Error comparing data completeness: {e}")

    # 4. Compare temperature ranges
    print("\n--- Temperature Range Comparison ---")
    try:
        mosdac_temp_valid = mosdac_ds[mosdac_temp_var].where(~np.isnan(mosdac_ds[mosdac_temp_var]))
        noaa_temp_valid = noaa_ds[noaa_temp_var].where(~np.isnan(noaa_ds[noaa_temp_var]))

        mosdac_range = [float(mosdac_temp_valid.min().values), float(mosdac_temp_valid.max().values)]
        noaa_range = [float(noaa_temp_valid.min().values), float(noaa_temp_valid.max().values)]

        print(f"MOSDAC temperature range: {mosdac_range}")
        print(f"NOAA temperature range: {noaa_range}")

        # Check if one is in Kelvin and the other in Celsius
        if abs(mosdac_range[0] - noaa_range[0]) > 250:
            print("NOTE: Datasets appear to use different temperature units")

            # If MOSDAC is in Kelvin and NOAA in Celsius
            if mosdac_range[0] > 200 and noaa_range[0] < 100:
                mosdac_celsius = [mosdac_range[0] - 273.15, mosdac_range[1] - 273.15]
                print(f"MOSDAC converted to Celsius: {mosdac_celsius}")

            # If NOAA is in Kelvin and MOSDAC in Celsius
            elif noaa_range[0] > 200 and mosdac_range[0] < 100:
                noaa_celsius = [noaa_range[0] - 273.15, noaa_range[1] - 273.15]
                print(f"NOAA converted to Celsius: {noaa_celsius}")
    except Exception as e:
        print(f"Error comparing temperature ranges: {e}")

    

# Example usage:
if __name__ == "__main__":
    # Update these paths to match your file locations
    mosdac_file =  r"C:\Users\Admin\RIYA PROJECT\DATASETS\mosdac_sst_cleaned.nc"
    noaa_file = r"C:\Users\Admin\RIYA PROJECT\DATASETS\noaa_sst.nc"

    # Load datasets
    mosdac_ds = load_dataset(mosdac_file, "MOSDAC")
    noaa_ds = load_dataset(noaa_file, "NOAA")

    # Compare datasets if both loaded successfully
    if mosdac_ds is not None and noaa_ds is not None:
        compare_datasets(mosdac_ds, noaa_ds)
        print("\n=== COMPARISON COMPLETE ===")
    else:
        print("\nCould not compare datasets due to loading errors")


=== LOADING MOSDAC DATASET ===
Successfully loaded: C:\Users\Admin\RIYA PROJECT\DATASETS\mosdac_sst_cleaned.nc
Dimensions: Frozen({'time': 278, 'y': 2816, 'x': 2805, 'dim_0': 1, 'dim_1': 2816, 'dim_2': 2805})
Variables: ['time', 'y', 'x', 'Latitude', 'Longitude', 'SST_WKL']
Found temperature variables: ['SST_WKL']
Temperature range: 276.5887451171875 to 311.41082763671875
Missing values: -71.28%

=== LOADING NOAA DATASET ===
Successfully loaded: C:\Users\Admin\RIYA PROJECT\DATASETS\noaa_sst.nc
Dimensions: Frozen({'time': 72, 'lat': 20, 'lon': 24})
Variables: ['time', 'lat', 'lon', 'sst']
Found temperature variables: ['sst']
Temperature range: 21.711427688598633 to 31.489999771118164
Missing values: 32.92%

=== COMPARING DATASETS ===
Comparing SST_WKL (MOSDAC) with sst (NOAA)

--- Spatial Coverage Comparison ---
MOSDAC latitude range: [-32768.0, 32767.0]
NOAA latitude range: [16.125, 20.875]
MOSDAC longitude range: [-715.0, 32767.0]
NOAA longitude range: [69.125, 74.875]

--- Data Comp

In [ ]:
'''
#ABOVE CODE'S TO BE REVIEWED parts:
2. Compare temporal coverage
    print("\n--- Temporal Coverage Comparison ---")
    try:
        # Assuming 'month' is available in both datasets
        if 'month' in mosdac_ds.dims and 'month' in noaa_ds.dims:
            mosdac_months = sorted(list(np.unique(mosdac_ds.month.values)))
            # Convert NOAA daily data to monthly means
        if 'time' in noaa_ds.coords:
            # Extract month values from datetime
            noaa_months = pd.DatetimeIndex(noaa_ds.time.values).month
            # Create a new coordinate for month
            noaa_ds = noaa_ds.assign_coords(month=("time", noaa_months))
            # Group by month and calculate mean
            noaa_monthly = noaa_ds.groupby("month").mean(dim="time")            

            print(f"MOSDAC months: {mosdac_months}")
            print(f"NOAA months: {noaa_months}")

            common_months = list(set(mosdac_ds.month.values).intersection(set(noaa_monthly.month.values)))
            if common_months:
                compare_month = common_months[0]  # Pick a month (e.g., January = 1)
                print(f"Comparing datasets for month {compare_month}")

            print(f"Common months: {sorted(list(common_months))}")
        else:
            print("Month dimension not found in both datasets")
    except Exception as e:
        print(f"Error comparing temporal coverage: {e}")
        
 5. Create a visualization comparing the two datasets
    print("\n--- Creating Comparison Visualization ---")

    try:
        # Select a common time point (use first time index as an example)
        common_time = noaa_ds.time[0]  # Adjust as needed
        print(f"Visualizing SST for {common_time.values}")

        # Select SST values at the chosen time
        mosdac_sst = mosdac_ds['SST_WKL'].isel(time=0)  # Modify based on actual structure
        noaa_sst = noaa_ds['sst'].sel(time=common_time)

        # Ensure the spatial grids align (regrid if necessary)
        if (mosdac_sst.shape != noaa_sst.shape):
            print("Warning: Grids do not match exactly; consider regridding!")

        # Create plots
        fig, axs = plt.subplots(1, 3, figsize=(18, 6))

        # Plot MOSDAC SST
        im1 = axs[0].pcolormesh(mosdac_ds.Longitude, mosdac_ds.Latitude, mosdac_sst, cmap='coolwarm', vmin=20, vmax=35)
        axs[0].set_title("MOSDAC SST")
        plt.colorbar(im1, ax=axs[0], label='SST (°C)')

        # Plot NOAA SST
        im2 = axs[1].pcolormesh(noaa_ds.lon, noaa_ds.lat, noaa_sst, cmap='coolwarm', vmin=20, vmax=35)
        axs[1].set_title("NOAA SST")
        plt.colorbar(im2, ax=axs[1], label='SST (°C)')

        # Compute and plot SST difference
        if mosdac_sst.shape == noaa_sst.shape:
            diff = mosdac_sst - noaa_sst
            im3 = axs[2].pcolormesh(noaa_ds.lon, noaa_ds.lat, diff, cmap='RdBu_r', vmin=-2, vmax=2)
            axs[2].set_title("MOSDAC - NOAA SST Difference")
            plt.colorbar(im3, ax=axs[2], label='Δ SST (°C)')
        else:
            axs[2].text(0.5, 0.5, "Grids do not match\nRegridding needed", ha='center', va='center', transform=axs[2].transAxes)
            axs[2].set_title("Difference (requires regridding)")

        plt.tight_layout()
        plt.show()
        plt.savefig('sst_comparison.png', dpi=300)
        print("Saved comparison plot as 'sst_comparison.png'")

    except Exception as e:
        print(f"Error creating visualization: {e}") '''

In [52]:
noaa = xr.open_dataset(r"C:\Users\Admin\RIYA PROJECT\DATASETS\noaa_konkan_sst.nc")
print(noaa['time'])

<xarray.DataArray 'time' (time: 73)>
array(['2019-01-01T00:00:00.000000000', '2019-02-01T00:00:00.000000000',
       '2019-03-01T00:00:00.000000000', '2019-04-01T00:00:00.000000000',
       '2019-05-01T00:00:00.000000000', '2019-06-01T00:00:00.000000000',
       '2019-07-01T00:00:00.000000000', '2019-08-01T00:00:00.000000000',
       '2019-09-01T00:00:00.000000000', '2019-10-01T00:00:00.000000000',
       '2019-11-01T00:00:00.000000000', '2019-12-01T00:00:00.000000000',
       '2020-01-01T00:00:00.000000000', '2020-02-01T00:00:00.000000000',
       '2020-03-01T00:00:00.000000000', '2020-04-01T00:00:00.000000000',
       '2020-05-01T00:00:00.000000000', '2020-06-01T00:00:00.000000000',
       '2020-07-01T00:00:00.000000000', '2020-08-01T00:00:00.000000000',
       '2020-09-01T00:00:00.000000000', '2020-10-01T00:00:00.000000000',
       '2020-11-01T00:00:00.000000000', '2020-12-01T00:00:00.000000000',
       '2021-01-01T00:00:00.000000000', '2021-02-01T00:00:00.000000000',
       '2021-0